In [1]:
import sys
sys.path.insert(0,'..')
import json

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path


# Log Extraction

We find all log directories that appear both in the data/save and data/log directories. We then extract arguments (args.json) of the run and last epoch macro/micro scores (last row of eval_valid.csv). If the eval_valid.csv file is missing we discard the run altogether. 

The code below assembles a pandas data frame of all run IDs that are names of both log and save directories.

In [2]:
LOGS = Path('../data/log')
SAVES = Path('../data/save')
MAXROWS = 300
TOPROWS = 10
pd.set_option('display.max_rows', MAXROWS)

In [3]:
logLabels = {x.name : x for x in LOGS.iterdir() if x.is_dir() and 'train' in str(x.name)}
saveLabels = {x.name : x for x in SAVES.iterdir() if x.is_dir()}
# reconcile logRun and saveRun Labels to match
logRunLabels = {x.name : L for L,D in logLabels.items() for x in D.iterdir() if x.is_dir()}
saveRunLabelsAll = {x.name : L for L,D in saveLabels.items() for x in D.iterdir() if x.is_dir() if x.name in logRunLabels.keys()}

validKeys = set(logRunLabels.keys()).intersection(set(saveRunLabelsAll.keys()))
logRunLabels = {k: v for k,v in logRunLabels.items() if k in validKeys}
saveRunLabels = {k: v for k,v in saveRunLabelsAll.items() if k in validKeys}

logRunPaths = {x.name :  x for L,D in logLabels.items() for x in D.iterdir() if x.is_dir() and x.name in validKeys }
saveRunPaths =  {x.name :  x for L,D in saveLabels.items() for x in D.iterdir() if x.is_dir() if x.name in validKeys}
runDF = pd.DataFrame([logRunLabels.keys(),logRunLabels.values(),logRunPaths.values()],columns=logRunLabels.keys(),index=['run','label','logPath']).T
#print(logRunLabels)
runDF = runDF.join(pd.DataFrame([saveRunPaths.keys(),saveRunPaths.values()],columns=logRunLabels.keys(),index=['runSave','savePath']).T,how='left').drop(columns = ['runSave'])
logLabels


{'ade_bert_train': PosixPath('../data/log/ade_bert_train'),
 'scierc_bert_train': PosixPath('../data/log/scierc_bert_train'),
 'scierc_rob_train': PosixPath('../data/log/scierc_rob_train'),
 'conll04_bert_train_lvl': PosixPath('../data/log/conll04_bert_train_lvl'),
 'conll04_rob_train_lvl': PosixPath('../data/log/conll04_rob_train_lvl'),
 'conll04_bert_train': PosixPath('../data/log/conll04_bert_train'),
 'docRed_rob_train_lvl': PosixPath('../data/log/docRed_rob_train_lvl'),
 'scierc_elec_train': PosixPath('../data/log/scierc_elec_train'),
 'conll04_elec_train': PosixPath('../data/log/conll04_elec_train'),
 'conll04_rob_train': PosixPath('../data/log/conll04_rob_train'),
 'ade_elec_train': PosixPath('../data/log/ade_elec_train'),
 'docRed_bert_train_lvl': PosixPath('../data/log/docRed_bert_train_lvl'),
 'ade_rob_train': PosixPath('../data/log/ade_rob_train')}

In [4]:
logEvalLabels = {x.name : x for x in LOGS.iterdir() if x.is_dir() and 'test' in str(x.name)}
logEvalRunLabels = {x.name : L for L,D in logEvalLabels.items() for x in D.iterdir() if x.is_dir()}
logEvalRunPaths = {x.name :  x for L,D in logEvalLabels.items() for x in D.iterdir() if x.is_dir() }
runEvalDF = pd.DataFrame([logEvalRunLabels.keys(),logEvalRunLabels.values(),logEvalRunPaths.values()],columns=logEvalRunLabels.keys(),index=['run','label','logPath']).T
runEvalDF

,run,label,logPath
2022-03-26_13.17.27.579408,2022-03-26_13.17.27.579408,conll04_bert_test,../data/log/conll04_bert_test/2022-03-26_13.17...
2022-03-26_13.09.08.704869,2022-03-26_13.09.08.704869,conll04_bert_test,../data/log/conll04_bert_test/2022-03-26_13.09...
2022-03-26_13.15.24.855735,2022-03-26_13.15.24.855735,conll04_bert_test,../data/log/conll04_bert_test/2022-03-26_13.15...
2022-03-26_13.18.15.750252,2022-03-26_13.18.15.750252,conll04_bert_test,../data/log/conll04_bert_test/2022-03-26_13.18...
2022-03-26_14.11.45.743272,2022-03-26_14.11.45.743272,conll04_rob_test,../data/log/conll04_rob_test/2022-03-26_14.11....
2022-03-27_14.52.01.153096,2022-03-27_14.52.01.153096,conll04_rob_test,../data/log/conll04_rob_test/2022-03-27_14.52....
2022-03-26_14.14.40.366734,2022-03-26_14.14.40.366734,conll04_rob_test,../data/log/conll04_rob_test/2022-03-26_14.14....
2022-03-26_14.16.29.477572,2022-03-26_14.16.29.477572,conll04_rob_test,../data/log/conll04_rob_test/2022-03-26_14.16....
2022-03-31_10.04.38.767961,2022-03-31_10.04.38.767961,conll04_rob_test,../data/log/conll04_rob_test/2022-03-31_10.04....
2022-03-26_14.15.40.228845,2022-03-26_14.15.40.228845,conll04_rob_test,../data/log/conll04_rob_test/2022-03-26_14.15....


For each of the log directories selected above, we extract the arguments (`args.json`) and the eval_valid.csv. We discard directories that don't contain eval_valid.csv (indicating incomplete runs). We create a data frame with all arguments, run info from the previous data frame and the scores of the last run. There is one row for each Run-ID. 

We show some of the columns below

In [5]:
argList= []
for run in runDF.itertuples():
    with open(run.logPath.joinpath('args.json')) as A:
        D = json.load(A)
    D['label']=run.label
    D['logPath'] = run.logPath
    D['savePath'] = run.savePath
    D['runID'] = run.run
    
    evPath = run.logPath.joinpath('eval_valid.csv')
    if evPath.exists():
        try:
            D.update(pd.read_csv(evPath,sep=';').iloc[-1].to_dict())
            argList.append(D)
        except:
            print('exception ==>',evPath)
#data/log/scierc_rob_train/2022-03-22_16.49.15.916860/eval_valid.csv
argDF = pd.DataFrame.from_dict(argList).drop(columns=['store_predictions',	'store_examples','tokenizer_path']).reset_index()
argDF[['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro','ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro','train_batch_size', 'epochs', 'neg_entity_count', 'neg_relation_count', 'lr', 'weight_decay',  'lowercase', 'model_path', 'rel_filter_threshold', 'prop_drop']]
display(argDF)


exception ==> ../data/log/scierc_bert_train/2022-03-31_09.44.48.114125/eval_valid.csv
exception ==> ../data/log/scierc_bert_train/2022-03-31_09.49.08.704424/eval_valid.csv
exception ==> ../data/log/scierc_bert_train/2022-03-31_10.03.55.903883/eval_valid.csv
exception ==> ../data/log/conll04_bert_train_lvl/2022-03-31_12.36.23.224056/eval_valid.csv
exception ==> ../data/log/conll04_bert_train_lvl/2022-03-31_19.16.53.301941/eval_valid.csv
exception ==> ../data/log/conll04_rob_train_lvl/2022-03-31_10.30.40.480713/eval_valid.csv
exception ==> ../data/log/conll04_rob_train_lvl/2022-03-31_10.31.33.581041/eval_valid.csv
exception ==> ../data/log/conll04_rob_train_lvl/2022-03-31_10.26.09.634505/eval_valid.csv
exception ==> ../data/log/conll04_rob_train_lvl/2022-03-31_10.41.47.528621/eval_valid.csv
exception ==> ../data/log/conll04_rob_train_lvl/2022-03-31_10.23.59.029523/eval_valid.csv
exception ==> ../data/log/conll04_rob_train_lvl/2022-03-31_10.36.02.704537/eval_valid.csv
exception ==> ../dat

,index,train_path,valid_path,save_path,init_eval,save_optimizer,train_log_iter,final_eval,train_batch_size,epochs,...,rel_nec_f1_micro,rel_nec_prec_macro,rel_nec_rec_macro,rel_nec_f1_macro,epoch,iteration,global_iteration,output_hidden_states,hidden_state_from_layer,f_score_beta
0,0,data/datasets/ade/ade_split_0_train.json,data/datasets/ade/ade_split_0_test.json,data/save/,False,False,100,False,4,20,...,81.199707,79.172611,83.333333,81.199707,20.0,0.0,19220.0,NaN,NaN,NaN
1,1,data/datasets/ade/ade_split_0_train.json,data/datasets/ade/ade_split_0_test.json,data/save/,False,False,100,False,4,20,...,81.049563,78.753541,83.483483,81.049563,20.0,0.0,19220.0,NaN,NaN,NaN
2,2,data/datasets/ade/ade_split_0_train.json,data/datasets/ade/ade_split_0_test.json,data/save/,False,False,100,False,4,20,...,82.753623,79.971989,85.735736,82.753623,20.0,0.0,19220.0,NaN,NaN,NaN
3,3,data/datasets/ade/ade_split_7_train.json,data/datasets/ade/ade_split_7_test.json,data/save/,False,False,100,False,4,20,...,80.433311,76.843467,84.375000,80.433311,20.0,0.0,19220.0,NaN,NaN,NaN
4,4,data/datasets/ade/ade_split_0_train.json,data/datasets/ade/ade_split_0_test.json,data/save/,False,False,100,False,4,20,...,83.819242,81.444759,86.336336,83.819242,20.0,0.0,19220.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,650,data/datasets/ade/ade_split_0_train.json,data/datasets/ade/ade_split_0_test.json,data/save/,False,False,100,False,4,20,...,75.609756,72.390110,79.129129,75.609756,20.0,0.0,19220.0,NaN,NaN,NaN
651,651,data/datasets/ade/ade_split_0_train.json,data/datasets/ade/ade_split_0_test.json,data/save/,False,False,100,False,4,20,...,81.129272,80.294118,81.981982,81.129272,20.0,0.0,19220.0,NaN,NaN,NaN
652,652,data/datasets/ade/ade_split_0_train.json,data/datasets/ade/ade_split_0_test.json,data/save/,False,False,100,False,4,20,...,81.355932,79.884226,82.882883,81.355932,20.0,0.0,19220.0,NaN,NaN,NaN
653,653,data/datasets/ade/ade_split_0_train.json,data/datasets/ade/ade_split_0_test.json,data/save/,False,False,100,False,4,20,...,81.001473,79.479769,82.582583,81.001473,20.0,0.0,19220.0,NaN,NaN,NaN


In [6]:
def best_runs(df=None,groupingLabel = 'label', 
    criteria = ['ner_f1_macro','rel_f1_macro',	'rel_nec_f1_macro'],
    topN=3):
    '''
    Function to extract best runs from a pandas dataframe of all runs 
    found in the standard directories.
    Argumnets:
        df: Pandas dataframe with results and arguments of each run
        groupingLable (default is 'label'): The column name by which to partition the data frame
            such partitioning will produce different best results for each groupingLable
        criteria: list of column headings denoting metrics which will determine the best runs
        topN: The numner of best N experiments to return per criterion per groupLabel
    '''
    res = pd.DataFrame()
    cols = ['index','criterion','label',	'runID',	'ner_f1_macro',	'rel_f1_macro',	'rel_nec_f1_macro',	'ner_f1_micro',	'rel_f1_micro',	'rel_nec_f1_micro','lr','weight_decay']

    for criterion in criteria:
        DF1=df.groupby('label').apply(lambda x : x.sort_values(by = criterion, ascending = False).head(topN).reset_index(drop = True))
        DF1['criterion']=criterion
        DF1=DF1.drop(columns=['config','types_path','train_path','valid_path','global_iteration','ner_prec_micro','ner_prec_macro','ner_rec_micro', 'ner_rec_macro','rel_prec_micro', 'rel_rec_micro','rel_rec_macro','rel_prec_macro', 'rel_rec_macro', 'rel_nec_prec_micro',
       'rel_nec_rec_micro','logPath','savePath','model_type'])
        #DF1.set_index('criterion', append=True, inplace=True)
        #DF1.reorder_levels([2, 0, 1])
        res=pd.concat([res,DF1],axis=0)
    res.set_index('criterion', append=True, inplace=True)
    res.index.names = ['label', 'seq', 'criterion']
    varCols = [col for col in res.columns if len(set(res[col])) > 1 or col in cols]
    res = res[varCols]

    #  
    return   res.reorder_levels(['criterion','label', 'seq']).drop(columns=['label'])

In [7]:
bestN = best_runs(argDF,
    criteria=['ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro','rel_nec_f1_micro'],
    topN=5)
df = bestN.reset_index()[['runID','label']]
#lookup = [runID : label for runID, label in df.iterrows()}
pd.DataFrame(bestN).to_csv('hyperparm_sel_bestN.csv')
bestN

index  train_batch_size  epochs  \
criterion        label            seq                                    
ner_f1_macro     ade_bert_train   0        4                 4      20   
                                  1        2                 4      20   
                                  2        7                 4      20   
                                  3       32                 4      20   
                                  4       20                 4      20   
...                                      ...               ...     ...   
rel_nec_f1_micro scierc_rob_train 0       79                 4      20   
                                  1       82                 4      20   
                                  2       77                 4      20   
                                  3       72                 4      20   
                                  4       73                 4      20   

                                       neg_entity_count  neg_relation_count  \
criterion        label            seq                                         
ner_f1_macro     ade_bert_train   0                 100                 100   
                                  1                 100                 100   
                                  2                 100                 100   
                                  3                 100                 100   
                                  4                 100                 100   
...                                                 ...                 ...   
rel_nec_f1_micro scierc_rob_train 0                 100                 100   
                                  1                 100                 100   
                                  2                 100                 100   
                                  3                 100                 100   
                                  4                 100                 100   

                                            lr  weight_decay  \
criterion        label            seq                          
ner_f1_macro     ade_bert_train   0    0.00001          0.02   
                                  1    0.00005          0.01   
                                  2    0.00001          0.02   
                                  3    0.00002          0.02   
                                  4    0.00005          0.00   
...                                        ...           ...   
rel_nec_f1_micro scierc_rob_train 0    0.00005          0.20   
                                  1    0.00006          0.20   
                                  2    0.00005          0.15   
                                  3    0.00005          0.10   
                                  4    0.00005          0.15   

                                                           model_path  \
criterion        label            seq                                   
ner_f1_macro     ade_bert_train   0    allenai/scibert_scivocab_cased   
                                  1    allenai/scibert_scivocab_cased   
                                  2    allenai/scibert_scivocab_cased   
                                  3    allenai/scibert_scivocab_cased   
                                  4    allenai/scibert_scivocab_cased   
...                                                               ...   
rel_nec_f1_micro scierc_rob_train 0       allenai/biomed_roberta_base   
                                  1       allenai/biomed_roberta_base   
                                  2       allenai/biomed_roberta_base   
                                  3       allenai/biomed_roberta_base   
                                  4       allenai/biomed_roberta_base   

                                       eval_batch_size  rel_filter_threshold  \
criterion        label            seq                                          
ner_f1_macro     ade_bert_train   0                  1                   0.4   
                         

In [8]:
recentSaveRunIDs  = sorted([x.name  for _,D in saveLabels.items() for x in D.iterdir() if x.is_dir() ])[-5:]
bestN_runIDs = set(bestN.runID)
print(recentSaveRunIDs)
saveDirsAll = [x  for _,D in saveLabels.items() for x in D.iterdir() if x.is_dir() ]
saveDirsXXX = [x  for _,D in saveLabels.items() for x in D.iterdir() if x.is_dir() and x.name not in bestN_runIDs  and x.name < recentSaveRunIDs[0]]
saveRidsXXX = sorted([x.name  for x in saveDirsXXX])
print('All save directories:',len(saveDirsAll))
print('Save dirs to remove: ',len(saveDirsXXX))
print('Save Directories to keep:',len(set(saveDirsAll).difference(set(saveDirsXXX))))
print('latest directories:' )
print('maximum deleted ID:', saveRidsXXX[-1] )

['2022-04-05_07.04.03.787270', '2022-04-05_09.47.37.728639', '2022-04-05_12.47.34.061412', '2022-04-05_15.37.34.269938', '2022-04-05_18.32.01.569345']
All save directories: 723
Save dirs to remove:  616
Save Directories to keep: 107
latest directories:
maximum deleted ID: 2022-04-04_20.22.18.432822


In [9]:
sorted(list(set(saveDirsAll).difference(set(saveDirsXXX))),key=lambda x : x.name)[-25:]

[PosixPath('../data/save/conll04_bert_train_lvl/2022-03-31_11.41.20.471429'),
 PosixPath('../data/save/conll04_bert_train_lvl/2022-03-31_11.50.34.017874'),
 PosixPath('../data/save/conll04_bert_train_lvl/2022-03-31_12.07.41.216129'),
 PosixPath('../data/save/conll04_bert_train_lvl/2022-03-31_16.19.15.856440'),
 PosixPath('../data/save/conll04_bert_train_lvl/2022-03-31_17.06.20.070977'),
 PosixPath('../data/save/conll04_bert_train_lvl/2022-03-31_17.30.03.393484'),
 PosixPath('../data/save/docRed_bert_train_lvl/2022-04-02_15.54.11.598914'),
 PosixPath('../data/save/docRed_bert_train_lvl/2022-04-03_15.16.37.215988'),
 PosixPath('../data/save/docRed_bert_train_lvl/2022-04-03_18.02.08.888896'),
 PosixPath('../data/save/docRed_bert_train_lvl/2022-04-04_02.30.54.745259'),
 PosixPath('../data/save/docRed_bert_train_lvl/2022-04-04_04.33.41.768240'),
 PosixPath('../data/save/docRed_bert_train_lvl/2022-04-04_06.37.09.385758'),
 PosixPath('../data/save/docRed_bert_train_lvl/2022-04-04_08.40.39.096

In [10]:
[str(d) for d in sorted(saveDirsXXX,key=lambda x : x.name)]


['../data/save/scierc_bert_train/2022-03-22_10.17.46.145076',
 '../data/save/scierc_bert_train/2022-03-22_10.46.00.854943',
 '../data/save/scierc_rob_train/2022-03-22_13.52.36.286048',
 '../data/save/scierc_rob_train/2022-03-22_14.31.14.577274',
 '../data/save/scierc_rob_train/2022-03-22_16.14.48.128661',
 '../data/save/scierc_rob_train/2022-03-22_16.16.10.286131',
 '../data/save/scierc_rob_train/2022-03-22_16.17.36.028741',
 '../data/save/scierc_rob_train/2022-03-22_16.18.08.564927',
 '../data/save/scierc_rob_train/2022-03-22_16.49.15.916860',
 '../data/save/scierc_rob_train/2022-03-22_17.36.49.251748',
 '../data/save/scierc_rob_train/2022-03-22_17.58.14.707227',
 '../data/save/scierc_elec_train/2022-03-23_09.43.18.960435',
 '../data/save/scierc_elec_train/2022-03-23_11.23.18.939754',
 '../data/save/scierc_rob_train/2022-03-23_18.28.13.297036',
 '../data/save/conll04_bert_train/2022-03-24_10.07.59.138914',
 '../data/save/conll04_bert_train/2022-03-24_10.11.21.546339',
 '../data/save/c

In [11]:
def best_run(df=None,groupingLabel = 'label', 
    maxMetrics = ['ner_f1_macro','rel_f1_macro',	'rel_nec_f1_macro']):
    '''
    Function to extract best runs from a pandas dataframe of all runs 
    found in the standard directories. 
    '''
    res = pd.DataFrame()
    for metric in maxMetrics:
        idx=df.groupby(by='label')[metric].idxmax()
        df1 = df.loc[idx,['label','runID']+maxMetrics]
        df1['criterion'] = metric
        res = pd.concat([res,df1],axis=0)
    return res

## Best Runs - Training
The best runs are identified below. The label is the one assigned to each run that tracks dataset and model type. The metrics shown are those based on which we make the choice of best run. The column "maximize" indicates which of the list of metrics was used to select the row. 

In [12]:
best = best_run(argDF,maxMetrics=['ner_f1_macro','rel_f1_macro',	'rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro'])
best

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,criterion
4,ade_bert_train,2022-03-27_09.07.07.554048,91.224258,83.819242,83.819242,90.934192,83.819242,83.819242,ner_f1_macro
613,ade_elec_train,2022-03-28_12.03.19.494299,90.838738,81.196013,81.196013,90.461538,81.196013,81.196013,ner_f1_macro
632,ade_rob_train,2022-03-27_22.47.55.820777,91.289223,82.532751,82.532751,90.942529,82.532751,82.532751,ner_f1_macro
162,conll04_bert_train,2022-03-24_17.03.33.337513,85.760877,70.471580,70.471580,88.751406,68.944099,68.944099,ner_f1_macro
90,conll04_bert_train_lvl,2022-03-31_17.30.03.393484,83.373162,72.543964,72.229003,86.737549,71.449925,71.150972,ner_f1_macro
274,conll04_elec_train,2022-03-26_10.39.43.538891,73.744530,62.670404,62.670404,77.066229,61.363636,61.363636,ner_f1_macro
570,conll04_rob_train,2022-03-25_10.31.25.567663,83.998313,67.612002,67.612002,87.148373,67.365269,67.365269,ner_f1_macro
91,conll04_rob_train_lvl,2022-03-31_10.44.39.925787,80.702417,68.092642,67.882115,84.713730,65.853659,65.548780,ner_f1_macro
617,docRed_bert_train_lvl,2022-04-04_18.41.50.524935,99.803744,64.975845,64.975845,99.857737,90.013720,90.013720,ner_f1_macro
214,docRed_rob_train_lvl,2022-04-04_20.29.00.322642,86.264739,22.630432,22.313493,87.547632,59.210526,58.947368,ner_f1_macro


We now look at the columns of this dataframe and identify colummn names that have more than one value. If all the rows have the same value in a column, then the column is not interesting in terms of hyperparameter selection. We then select most significant columns from the original dataframe for the experiments that appear in the best-list.

In [13]:
varCols = [col for col in argDF.columns if len(set(argDF[col])) > 1 ]
lmt = varCols.index('label') # we discard anything on the right of column label (addidional metrics)
#print(varCols)
df = argDF.iloc[list(set(best.index))][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]]\
    .drop(columns=['config','model_type','index','neg_entity_count',	'neg_relation_count','types_path','final_eval','train_path','valid_path'])
df.sort_values(['label','runID'])

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,lr,weight_decay,lowercase,model_path,eval_batch_size,rel_filter_threshold,prop_drop
4,ade_bert_train,2022-03-27_09.07.07.554048,91.224258,83.819242,83.819242,90.934192,83.819242,83.819242,4,20,0.00001,0.020,False,allenai/scibert_scivocab_cased,1,0.40,0.10
613,ade_elec_train,2022-03-28_12.03.19.494299,90.838738,81.196013,81.196013,90.461538,81.196013,81.196013,4,20,0.00005,0.020,False,kamalkraj/bioelectra-base-discriminator-pubmed,1,0.40,0.10
599,ade_elec_train,2022-03-28_13.38.42.558541,90.062642,81.231231,81.231231,89.786444,81.231231,81.231231,4,20,0.00005,0.010,False,kamalkraj/bioelectra-base-discriminator-pubmed,1,0.40,0.10
632,ade_rob_train,2022-03-27_22.47.55.820777,91.289223,82.532751,82.532751,90.942529,82.532751,82.532751,4,20,0.00005,0.010,False,allenai/biomed_roberta_base,1,0.40,0.20
121,conll04_bert_train,2022-03-24_16.56.59.416385,85.677381,73.202166,73.202166,88.651685,71.889401,71.889401,4,20,0.00005,0.100,False,bert-base-cased,1,0.40,0.20
162,conll04_bert_train,2022-03-24_17.03.33.337513,85.760877,70.471580,70.471580,88.751406,68.944099,68.944099,4,20,0.00005,0.100,False,bert-base-cased,1,0.50,0.00
195,conll04_bert_train,2022-03-24_19.12.06.248497,83.286697,73.041919,73.041919,87.443439,72.063492,72.063492,4,20,0.00005,0.005,False,bert-base-cased,1,0.50,0.20
145,conll04_bert_train,2022-03-24_19.30.46.210486,83.071676,73.270185,72.950185,86.800895,71.964018,71.664168,4,20,0.00005,0.000,False,bert-base-cased,1,0.40,0.20
83,conll04_bert_train_lvl,2022-03-31_11.50.34.017874,83.062654,71.048530,71.048530,87.138810,69.820555,69.820555,4,20,0.00005,0.100,False,bert-base-cased,1,0.50,0.20
87,conll04_bert_train_lvl,2022-03-31_16.19.15.856440,83.095939,73.189696,73.189696,86.711712,71.828666,71.828666,4,20,0.00005,0.100,False,bert-base-cased,1,0.60,0.20


## Best/Mean Runs - Testing

We pick the best weights from each of the selected training runs above. We run a test for each and report the best run.

In [14]:
argEvalList= []
for run in runEvalDF.itertuples():
    with open(run.logPath.joinpath('args.json')) as A:
        D = json.load(A)
    D['label']=run.label
    D['logPath'] = run.logPath
    #D['savePath'] = run.savePath
    D['runID'] = run.run
    
    evPath = run.logPath.joinpath('eval_test.csv')
    if evPath.exists():
        try:
            D.update(pd.read_csv(evPath,sep=';').iloc[-1].to_dict())
            argEvalList.append(D)
        except:
            print('exception ==>',evPath)

argEvalDF = pd.DataFrame.from_dict(argEvalList).reset_index()
argEvalDF[['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro','ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro','model_path', 'rel_filter_threshold']]


exception ==> ../data/log/conll04_rob_test/2022-03-27_14.52.01.153096/eval_test.csv
exception ==> ../data/log/conll04_rob_test/2022-03-31_10.04.38.767961/eval_test.csv


,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,model_path,rel_filter_threshold
0,conll04_bert_test,2022-03-26_13.17.27.579408,85.789502,73.091352,72.824686,88.642659,72.040302,71.788413,data/models/bert/conll04/2022-03-24_19.12.06.2...,0.5
1,conll04_bert_test,2022-03-26_13.09.08.704869,85.037992,69.378081,69.121670,87.824897,68.064118,67.817509,data/models/bert/conll04/2022-03-24_16.56.59.4...,0.5
2,conll04_bert_test,2022-03-26_13.15.24.855735,85.308104,68.993104,68.733363,88.327206,67.821782,67.574257,data/models/bert/conll04/2022-03-24_17.03.33.3...,0.5
3,conll04_bert_test,2022-03-26_13.18.15.750252,84.703277,70.210056,69.960056,87.885767,68.905473,68.656716,data/models/bert/conll04/2022-03-24_19.30.46.2...,0.5
4,conll04_rob_test,2022-03-26_14.14.40.366734,85.239432,70.327951,69.815883,87.827285,68.948035,68.441065,data/models/roberta/conll04/2022-03-25_05.43.5...,0.5
5,conll04_rob_test,2022-03-26_14.16.29.477572,85.472489,69.167520,69.167520,88.132475,67.478685,67.478685,data/models/roberta/conll04/2022-03-25_15.07.1...,0.5
6,conll04_rob_test,2022-03-26_14.15.40.228845,84.253314,70.866954,70.866954,86.916320,69.444444,69.444444,data/models/roberta/conll04/2022-03-25_06.10.1...,0.5
7,docRed_bert_test_lvl,2022-04-01_13.18.25.606821,72.876355,4.426773,4.303077,76.036580,16.670322,16.538715,data/models/bert/docRed/2022-04-01_10.50.12.67...,0.5
8,docRed_bert_test_lvl,2022-04-02_18.43.26.670247,87.455293,31.041569,30.414249,88.300041,57.079841,56.352007,data/models/bert/docRed/2022-04-02_15.54.11.59...,0.5
9,docRed_bert_test_lvl,2022-04-04_13.15.51.696886,86.139266,32.703839,31.792824,87.046892,58.706777,57.821991,data/models/bert/docRed/2022-04-03_18.02.08.88...,0.5


### Display Mean of Test Runs by Label

In [42]:
# Averate results of all tested runs:
means= pd.DataFrame(argEvalDF).groupby(by='label').mean(['ner_f1_macro','rel_f1_macro',	'rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro'])
display(means[['ner_f1_macro','rel_f1_macro',	'rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']])


,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro
label,,,,,,
conll04_bert_test,85.209719,70.418148,70.159944,88.170132,69.207919,68.959224
conll04_rob_test,84.988412,70.120808,69.950119,87.625360,68.623721,68.454731
docRed_bert_test_lvl,82.156971,22.724061,22.170050,83.794504,44.152314,43.570904


### Display best Test Runs by Label

In [41]:
best = best_run(argEvalDF,maxMetrics=['ner_f1_macro','rel_f1_macro',	'rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro'])
#print(best)
varColsTest = [col for col in argEvalDF.columns if len(set(argEvalDF[col])) > 1 or col == 'label']
lmtTest = varColsTest.index('label') # we discard anything on the right of column label (addidional metrics)
#print(varCols)
#argEvalDF.iloc[list(set(best.index))]
df = argEvalDF.iloc[list(set(best.index))][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varColsTest[:lmtTest]].drop(columns=['tokenizer_path'])
df = df.sort_values(['label','runID'])
df[['label'	,'runID',	'ner_f1_macro',	'rel_f1_macro',	'rel_nec_f1_macro'	,'ner_f1_micro',	'rel_f1_micro',	'rel_nec_f1_micro']]

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro
0,conll04_bert_test,2022-03-26_13.17.27.579408,85.789502,73.091352,72.824686,88.642659,72.040302,71.788413
6,conll04_rob_test,2022-03-26_14.15.40.228845,84.253314,70.866954,70.866954,86.916320,69.444444,69.444444
5,conll04_rob_test,2022-03-26_14.16.29.477572,85.472489,69.167520,69.167520,88.132475,67.478685,67.478685
8,docRed_bert_test_lvl,2022-04-02_18.43.26.670247,87.455293,31.041569,30.414249,88.300041,57.079841,56.352007
9,docRed_bert_test_lvl,2022-04-04_13.15.51.696886,86.139266,32.703839,31.792824,87.046892,58.706777,57.821991


## CoNLL04 - BERT Hyperparameter Search - Chronological Sequence

In [17]:
df = argDF.loc[argDF.label == 'conll04_bert_train'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase','model_path'])
pd.DataFrame(df).to_csv('hyperparm_sel_conll04_bert_train.csv')
df.sort_values('runID')


,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
113,conll04_bert_train,2022-03-24_10.11.21.546339,74.781620,55.913825,55.660661,79.872204,55.076142,54.822335,4,20,100,100,0.000005,0.100,1,0.4,0.0
139,conll04_bert_train,2022-03-24_10.17.07.397146,66.974673,43.934888,43.934888,73.276776,43.574051,43.574051,4,20,100,100,0.000005,0.100,1,0.4,0.1
187,conll04_bert_train,2022-03-24_10.22.57.368149,66.277413,44.126256,44.126256,72.949947,43.964422,43.964422,4,20,100,100,0.000005,0.100,1,0.4,0.2
175,conll04_bert_train,2022-03-24_10.29.25.335308,69.179068,45.109440,45.109440,76.002197,45.178336,45.178336,4,20,100,100,0.000005,0.100,1,0.5,0.0
116,conll04_bert_train,2022-03-24_10.36.00.136194,68.785280,49.930470,49.930470,75.184017,50.194049,50.194049,4,20,100,100,0.000005,0.100,1,0.5,0.1
119,conll04_bert_train,2022-03-24_10.42.34.301900,68.654684,46.802987,46.521296,74.851912,46.357616,46.092715,4,20,100,100,0.000005,0.100,1,0.5,0.2
151,conll04_bert_train,2022-03-24_10.48.56.001071,72.806643,51.894814,51.894814,78.850325,51.750000,51.750000,4,20,100,100,0.000005,0.050,1,0.4,0.0
125,conll04_bert_train,2022-03-24_10.55.08.682239,74.892555,56.301172,56.301172,80.632842,55.629139,55.629139,4,20,100,100,0.000005,0.050,1,0.4,0.1
192,conll04_bert_train,2022-03-24_11.01.42.482450,67.870696,43.257922,43.257922,73.364731,42.044135,42.044135,4,20,100,100,0.000005,0.050,1,0.4,0.2
105,conll04_bert_train,2022-03-24_11.08.27.152037,76.396685,59.023859,59.023859,81.803368,58.487874,58.487874,4,20,100,100,0.000005,0.050,1,0.5,0.0


## CoNLL04 - BERT Hyperparameter Search - Order by Ner-F1-Macro

In [18]:
df.sort_values('ner_f1_macro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
165,conll04_bert_train,2022-03-24_18.15.34.313136,83.924992,69.289401,68.976901,87.379740,68.168168,67.867868,4,20,100,100,0.00005,0.010,1,0.4,0.2
101,conll04_bert_train,2022-03-24_18.40.32.805603,84.020695,72.182235,72.182235,87.576602,71.056662,71.056662,4,20,100,100,0.00005,0.005,1,0.4,0.0
155,conll04_bert_train,2022-03-24_18.46.45.347701,84.370549,72.770703,72.770703,87.744824,71.257485,71.257485,4,20,100,100,0.00005,0.005,1,0.4,0.1
156,conll04_bert_train,2022-03-24_17.42.54.078069,84.446644,72.039514,72.039514,87.626547,70.551181,70.551181,4,20,100,100,0.00005,0.050,1,0.5,0.0
186,conll04_bert_train,2022-03-24_17.23.07.535948,84.474043,69.968245,69.650785,87.914559,68.181818,67.878788,4,20,100,100,0.00005,0.050,1,0.4,0.0
106,conll04_bert_train,2022-03-24_18.28.06.063987,84.570562,71.389198,71.389198,87.752809,70.109546,70.109546,4,20,100,100,0.00005,0.010,1,0.5,0.1
100,conll04_bert_train,2022-03-24_19.36.49.329244,84.968826,71.136353,70.821392,88.000000,69.871795,69.551282,4,20,100,100,0.00005,0.000,1,0.5,0.0
141,conll04_bert_train,2022-03-24_16.43.39.922458,85.289893,69.692948,69.692948,88.750707,68.923077,68.923077,4,20,100,100,0.00005,0.100,1,0.4,0.0
121,conll04_bert_train,2022-03-24_16.56.59.416385,85.677381,73.202166,73.202166,88.651685,71.889401,71.889401,4,20,100,100,0.00005,0.100,1,0.4,0.2
162,conll04_bert_train,2022-03-24_17.03.33.337513,85.760877,70.471580,70.471580,88.751406,68.944099,68.944099,4,20,100,100,0.00005,0.100,1,0.5,0.0


## CoNLL04 - BERT Hyperparameter Search - Order by Rel-F1-Macro

In [19]:
df.sort_values('rel_f1_macro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
153,conll04_bert_train,2022-03-24_18.59.13.782335,83.277073,72.074899,72.074899,86.764706,70.906200,70.906200,4,20,100,100,0.00005,0.005,1,0.5,0.0
101,conll04_bert_train,2022-03-24_18.40.32.805603,84.020695,72.182235,72.182235,87.576602,71.056662,71.056662,4,20,100,100,0.00005,0.005,1,0.4,0.0
97,conll04_bert_train,2022-03-24_18.34.19.999196,83.809605,72.237624,71.922663,87.107623,70.917574,70.606532,4,20,100,100,0.00005,0.010,1,0.5,0.2
166,conll04_bert_train,2022-03-24_22.14.36.474140,82.722604,72.254512,72.254512,86.557005,71.090047,71.090047,4,20,100,100,0.00010,0.005,1,0.5,0.1
138,conll04_bert_train,2022-03-24_20.15.25.970327,82.186638,72.352607,72.022028,86.166008,71.632330,71.315372,4,20,100,100,0.00010,0.100,1,0.5,0.0
172,conll04_bert_train,2022-03-24_21.48.43.527680,83.771658,72.453193,72.135732,87.323944,71.406491,71.097372,4,20,100,100,0.00010,0.005,1,0.4,0.0
155,conll04_bert_train,2022-03-24_18.46.45.347701,84.370549,72.770703,72.770703,87.744824,71.257485,71.257485,4,20,100,100,0.00005,0.005,1,0.4,0.1
195,conll04_bert_train,2022-03-24_19.12.06.248497,83.286697,73.041919,73.041919,87.443439,72.063492,72.063492,4,20,100,100,0.00005,0.005,1,0.5,0.2
121,conll04_bert_train,2022-03-24_16.56.59.416385,85.677381,73.202166,73.202166,88.651685,71.889401,71.889401,4,20,100,100,0.00005,0.100,1,0.4,0.2
145,conll04_bert_train,2022-03-24_19.30.46.210486,83.071676,73.270185,72.950185,86.800895,71.964018,71.664168,4,20,100,100,0.00005,0.000,1,0.4,0.2


## CoNLL04 - BERT Hyperparameter Search - Order by Rel-Nec-F1-Macro (Joint Rel)

In [20]:
df.sort_values('rel_nec_f1_macro').tail(TOPROWS)


,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
138,conll04_bert_train,2022-03-24_20.15.25.970327,82.186638,72.352607,72.022028,86.166008,71.632330,71.315372,4,20,100,100,0.00010,0.100,1,0.5,0.0
156,conll04_bert_train,2022-03-24_17.42.54.078069,84.446644,72.039514,72.039514,87.626547,70.551181,70.551181,4,20,100,100,0.00005,0.050,1,0.5,0.0
153,conll04_bert_train,2022-03-24_18.59.13.782335,83.277073,72.074899,72.074899,86.764706,70.906200,70.906200,4,20,100,100,0.00005,0.005,1,0.5,0.0
172,conll04_bert_train,2022-03-24_21.48.43.527680,83.771658,72.453193,72.135732,87.323944,71.406491,71.097372,4,20,100,100,0.00010,0.005,1,0.4,0.0
101,conll04_bert_train,2022-03-24_18.40.32.805603,84.020695,72.182235,72.182235,87.576602,71.056662,71.056662,4,20,100,100,0.00005,0.005,1,0.4,0.0
166,conll04_bert_train,2022-03-24_22.14.36.474140,82.722604,72.254512,72.254512,86.557005,71.090047,71.090047,4,20,100,100,0.00010,0.005,1,0.5,0.1
155,conll04_bert_train,2022-03-24_18.46.45.347701,84.370549,72.770703,72.770703,87.744824,71.257485,71.257485,4,20,100,100,0.00005,0.005,1,0.4,0.1
145,conll04_bert_train,2022-03-24_19.30.46.210486,83.071676,73.270185,72.950185,86.800895,71.964018,71.664168,4,20,100,100,0.00005,0.000,1,0.4,0.2
195,conll04_bert_train,2022-03-24_19.12.06.248497,83.286697,73.041919,73.041919,87.443439,72.063492,72.063492,4,20,100,100,0.00005,0.005,1,0.5,0.2
121,conll04_bert_train,2022-03-24_16.56.59.416385,85.677381,73.202166,73.202166,88.651685,71.889401,71.889401,4,20,100,100,0.00005,0.100,1,0.4,0.2


## CoNLL04 - BERT Hyperparameter Search - Order by Ner-F1-Micro

In [21]:
df.sort_values('ner_f1_micro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
110,conll04_bert_train,2022-03-24_20.59.00.634490,83.792455,69.587439,69.259570,87.492861,68.656716,68.325041,4,20,100,100,0.00010,0.050,1,0.5,0.1
101,conll04_bert_train,2022-03-24_18.40.32.805603,84.020695,72.182235,72.182235,87.576602,71.056662,71.056662,4,20,100,100,0.00005,0.005,1,0.4,0.0
156,conll04_bert_train,2022-03-24_17.42.54.078069,84.446644,72.039514,72.039514,87.626547,70.551181,70.551181,4,20,100,100,0.00005,0.050,1,0.5,0.0
155,conll04_bert_train,2022-03-24_18.46.45.347701,84.370549,72.770703,72.770703,87.744824,71.257485,71.257485,4,20,100,100,0.00005,0.005,1,0.4,0.1
106,conll04_bert_train,2022-03-24_18.28.06.063987,84.570562,71.389198,71.389198,87.752809,70.109546,70.109546,4,20,100,100,0.00005,0.010,1,0.5,0.1
186,conll04_bert_train,2022-03-24_17.23.07.535948,84.474043,69.968245,69.650785,87.914559,68.181818,67.878788,4,20,100,100,0.00005,0.050,1,0.4,0.0
100,conll04_bert_train,2022-03-24_19.36.49.329244,84.968826,71.136353,70.821392,88.000000,69.871795,69.551282,4,20,100,100,0.00005,0.000,1,0.5,0.0
121,conll04_bert_train,2022-03-24_16.56.59.416385,85.677381,73.202166,73.202166,88.651685,71.889401,71.889401,4,20,100,100,0.00005,0.100,1,0.4,0.2
141,conll04_bert_train,2022-03-24_16.43.39.922458,85.289893,69.692948,69.692948,88.750707,68.923077,68.923077,4,20,100,100,0.00005,0.100,1,0.4,0.0
162,conll04_bert_train,2022-03-24_17.03.33.337513,85.760877,70.471580,70.471580,88.751406,68.944099,68.944099,4,20,100,100,0.00005,0.100,1,0.5,0.0


## CoNLL04 - BERT Hyperparameter Search - Order by Rel-Nec-F1-Micro (Joint Rel)

In [22]:
df.sort_values('rel_nec_f1_micro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
184,conll04_bert_train,2022-03-24_16.50.16.408540,82.800037,72.027338,71.702135,86.419753,70.928463,70.624049,4,20,100,100,0.00005,0.100,1,0.4,0.1
153,conll04_bert_train,2022-03-24_18.59.13.782335,83.277073,72.074899,72.074899,86.764706,70.906200,70.906200,4,20,100,100,0.00005,0.005,1,0.5,0.0
101,conll04_bert_train,2022-03-24_18.40.32.805603,84.020695,72.182235,72.182235,87.576602,71.056662,71.056662,4,20,100,100,0.00005,0.005,1,0.4,0.0
166,conll04_bert_train,2022-03-24_22.14.36.474140,82.722604,72.254512,72.254512,86.557005,71.090047,71.090047,4,20,100,100,0.00010,0.005,1,0.5,0.1
172,conll04_bert_train,2022-03-24_21.48.43.527680,83.771658,72.453193,72.135732,87.323944,71.406491,71.097372,4,20,100,100,0.00010,0.005,1,0.4,0.0
155,conll04_bert_train,2022-03-24_18.46.45.347701,84.370549,72.770703,72.770703,87.744824,71.257485,71.257485,4,20,100,100,0.00005,0.005,1,0.4,0.1
138,conll04_bert_train,2022-03-24_20.15.25.970327,82.186638,72.352607,72.022028,86.166008,71.632330,71.315372,4,20,100,100,0.00010,0.100,1,0.5,0.0
145,conll04_bert_train,2022-03-24_19.30.46.210486,83.071676,73.270185,72.950185,86.800895,71.964018,71.664168,4,20,100,100,0.00005,0.000,1,0.4,0.2
121,conll04_bert_train,2022-03-24_16.56.59.416385,85.677381,73.202166,73.202166,88.651685,71.889401,71.889401,4,20,100,100,0.00005,0.100,1,0.4,0.2
195,conll04_bert_train,2022-03-24_19.12.06.248497,83.286697,73.041919,73.041919,87.443439,72.063492,72.063492,4,20,100,100,0.00005,0.005,1,0.5,0.2


## CoNLL Roberta Hyperparameter Search - Chronological Sequence

In [23]:

df = argDF.loc[argDF.label == 'conll04_rob_train'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase','model_path'])
pd.DataFrame(df).to_csv('hyperparm_sel_conll04_rob_train.csv')
df.sort_values('runID')


,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
470,conll04_rob_train,2022-03-24_22.30.16.672979,49.020456,6.082837,5.976737,53.082353,5.754141,5.666957,4,20,100,100,0.000005,0.050,1,0.40,0.00
505,conll04_rob_train,2022-03-24_22.36.17.249570,49.261654,5.251629,5.176582,52.831985,4.740406,4.665162,4,20,100,100,0.000005,0.050,1,0.40,0.10
468,conll04_rob_train,2022-03-24_22.42.12.250809,50.907387,7.575784,7.466495,55.051936,6.324868,6.229037,4,20,100,100,0.000005,0.050,1,0.40,0.20
539,conll04_rob_train,2022-03-24_22.48.07.381242,54.894374,8.255678,8.046254,60.311089,8.688525,8.524590,4,20,100,100,0.000005,0.050,1,0.50,0.00
401,conll04_rob_train,2022-03-24_22.54.00.687065,51.427810,9.414703,9.257840,55.626134,8.841733,8.664898,4,20,100,100,0.000005,0.050,1,0.50,0.10
529,conll04_rob_train,2022-03-24_22.59.56.515944,50.618779,7.243250,7.048128,54.494644,6.218905,6.094527,4,20,100,100,0.000005,0.050,1,0.50,0.20
391,conll04_rob_train,2022-03-24_23.05.49.532944,52.507023,10.166205,10.002271,57.211538,10.360656,10.229508,4,20,100,100,0.000005,0.010,1,0.40,0.00
548,conll04_rob_train,2022-03-24_23.12.00.877955,53.084029,6.314160,6.219148,57.947494,6.445312,6.347656,4,20,100,100,0.000005,0.010,1,0.40,0.10
417,conll04_rob_train,2022-03-24_23.19.10.895127,49.220069,3.950562,3.857104,53.732763,4.543542,4.459403,4,20,100,100,0.000005,0.010,1,0.40,0.20
518,conll04_rob_train,2022-03-24_23.26.05.646746,53.639169,9.854601,9.854601,60.215054,10.460251,10.460251,4,20,100,100,0.000005,0.010,1,0.50,0.00


## CoNLL Roberta Hyperparameter Search - Ordered by NER-F1-Macro

In [24]:
df.sort_values('ner_f1_macro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
397,conll04_rob_train,2022-03-25_16.52.57.783758,82.931974,69.360816,69.133543,86.568849,68.018721,67.706708,4,20,100,100,0.00006,0.010,1,0.40,0.00
440,conll04_rob_train,2022-03-25_12.57.11.825786,82.962312,66.551307,66.551307,86.346048,65.263158,65.263158,4,20,100,100,0.00004,0.005,1,0.45,0.25
528,conll04_rob_train,2022-03-25_23.03.23.891473,82.967249,65.013107,65.013107,86.103233,63.711002,63.711002,4,20,100,100,0.00006,0.000,1,0.40,0.20
495,conll04_rob_train,2022-03-25_21.35.13.339052,82.995811,67.893634,67.332687,86.081610,66.666667,66.028708,4,20,100,100,0.00006,0.005,1,0.50,0.30
463,conll04_rob_train,2022-03-25_17.21.35.953860,83.018108,67.160662,66.932090,86.381323,66.568915,66.275660,4,20,100,100,0.00006,0.010,1,0.40,0.30
435,conll04_rob_train,2022-03-25_06.23.24.400353,83.172110,69.293786,68.643379,86.741573,67.828843,67.194929,4,20,100,100,0.00005,0.000,1,0.50,0.20
487,conll04_rob_train,2022-03-25_14.30.28.972095,83.176185,66.870073,66.870073,86.456606,65.785609,65.785609,4,20,100,100,0.00004,0.000,1,0.45,0.00
493,conll04_rob_train,2022-03-25_15.07.11.369668,83.397201,71.300103,71.300103,87.043189,70.889894,70.889894,4,20,100,100,0.00004,0.000,1,0.50,0.00
382,conll04_rob_train,2022-03-25_18.42.54.299054,83.587431,69.363824,69.363824,87.024609,67.987805,67.987805,4,20,100,100,0.00006,0.010,1,0.55,0.00
570,conll04_rob_train,2022-03-25_10.31.25.567663,83.998313,67.612002,67.612002,87.148373,67.365269,67.365269,4,20,100,100,0.00003,0.005,1,0.55,0.25


## CoNLL Roberta Hyperparameter Search - Ordered by Rel-F1-Macro

In [25]:
df.sort_values('rel_f1_macro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
402,conll04_rob_train,2022-03-25_04.49.45.779744,81.607086,69.667698,69.369191,85.476718,68.053492,67.756315,4,20,100,100,0.00005,0.010,1,0.50,0.00
513,conll04_rob_train,2022-03-25_21.42.01.583934,81.765605,69.751033,69.420455,85.234899,68.131868,67.817896,4,20,100,100,0.00006,0.005,1,0.55,0.00
545,conll04_rob_train,2022-03-25_23.29.54.566368,81.231363,69.817336,69.499876,85.361750,68.300153,67.993874,4,20,100,100,0.00006,0.000,1,0.45,0.10
502,conll04_rob_train,2022-03-25_12.49.46.725114,82.653014,70.156681,70.156681,85.682698,68.996960,68.996960,4,20,100,100,0.00004,0.005,1,0.45,0.15
547,conll04_rob_train,2022-03-25_13.19.51.478648,81.235319,70.213031,70.213031,85.398230,68.693009,68.693009,4,20,100,100,0.00004,0.005,1,0.50,0.00
550,conll04_rob_train,2022-03-25_17.07.14.234155,80.899818,70.215380,69.669392,84.885666,68.986384,68.381241,4,20,100,100,0.00006,0.010,1,0.40,0.20
379,conll04_rob_train,2022-03-25_17.59.01.982799,81.421050,70.275368,70.275368,85.067873,68.878357,68.878357,4,20,100,100,0.00006,0.010,1,0.45,0.30
493,conll04_rob_train,2022-03-25_15.07.11.369668,83.397201,71.300103,71.300103,87.043189,70.889894,70.889894,4,20,100,100,0.00004,0.000,1,0.50,0.00
535,conll04_rob_train,2022-03-25_05.43.53.077390,82.440380,71.407015,71.407015,86.189684,70.212766,70.212766,4,20,100,100,0.00005,0.005,1,0.50,0.20
486,conll04_rob_train,2022-03-25_06.10.18.550381,82.761402,71.518753,71.206253,86.237514,69.656203,69.357250,4,20,100,100,0.00005,0.000,1,0.50,0.00


## CoNLL Roberta Hyperparameter Search - Ordered by Rel-Nec-F1-Macro

In [26]:
df.sort_values('rel_nec_f1_macro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
513,conll04_rob_train,2022-03-25_21.42.01.583934,81.765605,69.751033,69.420455,85.234899,68.131868,67.817896,4,20,100,100,0.00006,0.005,1,0.55,0.00
545,conll04_rob_train,2022-03-25_23.29.54.566368,81.231363,69.817336,69.499876,85.361750,68.300153,67.993874,4,20,100,100,0.00006,0.000,1,0.45,0.10
522,conll04_rob_train,2022-03-25_18.06.53.745483,82.283583,69.566608,69.566608,85.554936,68.381241,68.381241,4,20,100,100,0.00006,0.010,1,0.50,0.00
550,conll04_rob_train,2022-03-25_17.07.14.234155,80.899818,70.215380,69.669392,84.885666,68.986384,68.381241,4,20,100,100,0.00006,0.010,1,0.40,0.20
502,conll04_rob_train,2022-03-25_12.49.46.725114,82.653014,70.156681,70.156681,85.682698,68.996960,68.996960,4,20,100,100,0.00004,0.005,1,0.45,0.15
547,conll04_rob_train,2022-03-25_13.19.51.478648,81.235319,70.213031,70.213031,85.398230,68.693009,68.693009,4,20,100,100,0.00004,0.005,1,0.50,0.00
379,conll04_rob_train,2022-03-25_17.59.01.982799,81.421050,70.275368,70.275368,85.067873,68.878357,68.878357,4,20,100,100,0.00006,0.010,1,0.45,0.30
486,conll04_rob_train,2022-03-25_06.10.18.550381,82.761402,71.518753,71.206253,86.237514,69.656203,69.357250,4,20,100,100,0.00005,0.000,1,0.50,0.00
493,conll04_rob_train,2022-03-25_15.07.11.369668,83.397201,71.300103,71.300103,87.043189,70.889894,70.889894,4,20,100,100,0.00004,0.000,1,0.50,0.00
535,conll04_rob_train,2022-03-25_05.43.53.077390,82.440380,71.407015,71.407015,86.189684,70.212766,70.212766,4,20,100,100,0.00005,0.005,1,0.50,0.20


## CoNLL Roberta Hyperparameter Search - Ordered by NER-F1-Micro

In [27]:
df.sort_values('ner_f1_micro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
543,conll04_rob_train,2022-03-25_20.33.09.982181,82.419102,69.473505,69.161005,86.429366,68.231612,67.918623,4,20,100,100,0.00006,0.005,1,0.45,0.00
582,conll04_rob_train,2022-03-25_20.47.15.293557,82.871541,68.290086,68.290086,86.455982,66.666667,66.666667,4,20,100,100,0.00006,0.005,1,0.45,0.20
487,conll04_rob_train,2022-03-25_14.30.28.972095,83.176185,66.870073,66.870073,86.456606,65.785609,65.785609,4,20,100,100,0.00004,0.000,1,0.45,0.00
416,conll04_rob_train,2022-03-25_12.13.32.237476,82.702744,66.166195,66.166195,86.552297,65.937500,65.937500,4,20,100,100,0.00003,0.000,1,0.55,0.15
397,conll04_rob_train,2022-03-25_16.52.57.783758,82.931974,69.360816,69.133543,86.568849,68.018721,67.706708,4,20,100,100,0.00006,0.010,1,0.40,0.00
465,conll04_rob_train,2022-03-25_21.48.51.042857,82.597646,69.111333,69.111333,86.580573,67.682927,67.682927,4,20,100,100,0.00006,0.005,1,0.55,0.10
435,conll04_rob_train,2022-03-25_06.23.24.400353,83.172110,69.293786,68.643379,86.741573,67.828843,67.194929,4,20,100,100,0.00005,0.000,1,0.50,0.20
382,conll04_rob_train,2022-03-25_18.42.54.299054,83.587431,69.363824,69.363824,87.024609,67.987805,67.987805,4,20,100,100,0.00006,0.010,1,0.55,0.00
493,conll04_rob_train,2022-03-25_15.07.11.369668,83.397201,71.300103,71.300103,87.043189,70.889894,70.889894,4,20,100,100,0.00004,0.000,1,0.50,0.00
570,conll04_rob_train,2022-03-25_10.31.25.567663,83.998313,67.612002,67.612002,87.148373,67.365269,67.365269,4,20,100,100,0.00003,0.005,1,0.55,0.25


## CoNLL Roberta Hyperparameter Search - Ordered by Rel-F1-Micro

In [28]:
df.sort_values('rel_f1_micro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
522,conll04_rob_train,2022-03-25_18.06.53.745483,82.283583,69.566608,69.566608,85.554936,68.381241,68.381241,4,20,100,100,0.00006,0.010,1,0.50,0.00
551,conll04_rob_train,2022-03-25_23.23.15.663822,82.097435,69.437696,68.887101,85.971831,68.535826,67.912773,4,20,100,100,0.00006,0.000,1,0.45,0.00
547,conll04_rob_train,2022-03-25_13.19.51.478648,81.235319,70.213031,70.213031,85.398230,68.693009,68.693009,4,20,100,100,0.00004,0.005,1,0.50,0.00
390,conll04_rob_train,2022-03-25_10.23.45.506984,81.696946,69.157120,69.157120,85.745856,68.789809,68.789809,4,20,100,100,0.00003,0.005,1,0.55,0.15
379,conll04_rob_train,2022-03-25_17.59.01.982799,81.421050,70.275368,70.275368,85.067873,68.878357,68.878357,4,20,100,100,0.00006,0.010,1,0.45,0.30
550,conll04_rob_train,2022-03-25_17.07.14.234155,80.899818,70.215380,69.669392,84.885666,68.986384,68.381241,4,20,100,100,0.00006,0.010,1,0.40,0.20
502,conll04_rob_train,2022-03-25_12.49.46.725114,82.653014,70.156681,70.156681,85.682698,68.996960,68.996960,4,20,100,100,0.00004,0.005,1,0.45,0.15
486,conll04_rob_train,2022-03-25_06.10.18.550381,82.761402,71.518753,71.206253,86.237514,69.656203,69.357250,4,20,100,100,0.00005,0.000,1,0.50,0.00
535,conll04_rob_train,2022-03-25_05.43.53.077390,82.440380,71.407015,71.407015,86.189684,70.212766,70.212766,4,20,100,100,0.00005,0.005,1,0.50,0.20
493,conll04_rob_train,2022-03-25_15.07.11.369668,83.397201,71.300103,71.300103,87.043189,70.889894,70.889894,4,20,100,100,0.00004,0.000,1,0.50,0.00


## CoNLL Roberta Hyperparameter Search - Ordered by Joint Rel-Nec-F1-Micro

In [29]:
df.sort_values('rel_nec_f1_micro').tail(TOPROWS)

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
508,conll04_rob_train,2022-03-25_15.49.32.429173,81.859468,69.166247,69.166247,85.524126,68.322981,68.322981,4,20,100,100,0.00004,0.000,1,0.55,0.15
550,conll04_rob_train,2022-03-25_17.07.14.234155,80.899818,70.215380,69.669392,84.885666,68.986384,68.381241,4,20,100,100,0.00006,0.010,1,0.40,0.20
522,conll04_rob_train,2022-03-25_18.06.53.745483,82.283583,69.566608,69.566608,85.554936,68.381241,68.381241,4,20,100,100,0.00006,0.010,1,0.50,0.00
547,conll04_rob_train,2022-03-25_13.19.51.478648,81.235319,70.213031,70.213031,85.398230,68.693009,68.693009,4,20,100,100,0.00004,0.005,1,0.50,0.00
390,conll04_rob_train,2022-03-25_10.23.45.506984,81.696946,69.157120,69.157120,85.745856,68.789809,68.789809,4,20,100,100,0.00003,0.005,1,0.55,0.15
379,conll04_rob_train,2022-03-25_17.59.01.982799,81.421050,70.275368,70.275368,85.067873,68.878357,68.878357,4,20,100,100,0.00006,0.010,1,0.45,0.30
502,conll04_rob_train,2022-03-25_12.49.46.725114,82.653014,70.156681,70.156681,85.682698,68.996960,68.996960,4,20,100,100,0.00004,0.005,1,0.45,0.15
486,conll04_rob_train,2022-03-25_06.10.18.550381,82.761402,71.518753,71.206253,86.237514,69.656203,69.357250,4,20,100,100,0.00005,0.000,1,0.50,0.00
535,conll04_rob_train,2022-03-25_05.43.53.077390,82.440380,71.407015,71.407015,86.189684,70.212766,70.212766,4,20,100,100,0.00005,0.005,1,0.50,0.20
493,conll04_rob_train,2022-03-25_15.07.11.369668,83.397201,71.300103,71.300103,87.043189,70.889894,70.889894,4,20,100,100,0.00004,0.000,1,0.50,0.00


## CoNLL Electra Hyperparameter Search

In [30]:
df = argDF.loc[argDF.label == 'conll04_elec_train'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase','model_path'])
pd.DataFrame(df).to_csv('hyperparm_sel_conll04_elec_train.csv')
df.sort_values('runID')

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
251,conll04_elec_train,2022-03-25_23.57.55.278306,54.340244,25.800590,25.800590,60.381143,24.887286,24.887286,4,20,100,100,0.000005,0.050,1,0.40,0.0
348,conll04_elec_train,2022-03-26_00.04.36.993327,43.566466,16.387229,16.228498,48.543689,16.116505,15.922330,4,20,100,100,0.000005,0.050,1,0.40,0.1
245,conll04_elec_train,2022-03-26_00.10.43.487585,50.194499,31.504380,31.296046,57.816504,30.223390,29.960578,4,20,100,100,0.000005,0.050,1,0.40,0.2
334,conll04_elec_train,2022-03-26_00.16.51.170918,46.704458,19.345334,18.651674,52.906667,19.392523,18.691589,4,20,100,100,0.000005,0.050,1,0.40,0.3
239,conll04_elec_train,2022-03-26_00.22.59.482583,54.554951,32.010306,32.010306,60.661578,29.723502,29.723502,4,20,100,100,0.000005,0.050,1,0.45,0.0
367,conll04_elec_train,2022-03-26_00.29.06.350760,49.936122,25.007245,24.077012,56.153051,25.482625,24.453024,4,20,100,100,0.000005,0.050,1,0.45,0.1
352,conll04_elec_train,2022-03-26_00.35.13.148708,52.014272,31.991596,31.451056,59.754142,32.300163,31.647635,4,20,100,100,0.000005,0.050,1,0.45,0.2
235,conll04_elec_train,2022-03-26_00.41.18.421922,52.474772,29.627902,29.627902,58.442872,29.638854,29.638854,4,20,100,100,0.000005,0.050,1,0.45,0.3
345,conll04_elec_train,2022-03-26_00.47.28.606614,43.050775,18.788695,18.547731,49.502982,18.221258,18.004338,4,20,100,100,0.000005,0.050,1,0.50,0.0
358,conll04_elec_train,2022-03-26_00.53.34.273972,52.561672,32.637919,32.637919,59.674882,32.611833,32.611833,4,20,100,100,0.000005,0.050,1,0.50,0.1


In [31]:
df = argDF.loc[argDF.label == 'scierc_bert_train'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase','model_path'])
pd.DataFrame(df).to_csv('hyperparm_sel_scierc_elec_train.csv')
df.sort_values('runID')

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
49,scierc_bert_train,2022-03-22_09.51.47.931037,70.308508,49.589145,38.588136,71.283582,50.054885,39.736553,4,20,100,100,0.00006,0.020,1,0.4,0.15
45,scierc_bert_train,2022-03-22_10.17.46.145076,69.027011,47.519995,39.019091,70.257038,45.942857,39.085714,4,20,100,100,0.00006,0.020,1,0.4,0.15
46,scierc_bert_train,2022-03-22_10.46.00.854943,67.227290,42.934329,30.853080,68.150897,45.979615,34.654587,4,20,100,100,0.00006,0.020,1,0.4,0.15
58,scierc_bert_train,2022-03-26_16.05.11.410843,71.440460,50.306124,41.055718,73.033033,50.222222,42.888889,4,20,100,100,0.00005,0.020,1,0.4,0.00
48,scierc_bert_train,2022-03-26_16.17.02.300171,72.068276,49.307465,39.706550,73.224044,49.720045,41.433371,4,20,100,100,0.00005,0.020,1,0.4,0.10
50,scierc_bert_train,2022-03-26_16.28.45.152161,71.644596,49.682922,39.928613,72.651107,50.000000,41.447368,4,20,100,100,0.00005,0.020,1,0.4,0.20
67,scierc_bert_train,2022-03-26_16.40.26.222403,70.742345,50.698155,41.427070,72.593501,50.756694,42.142026,4,20,100,100,0.00005,0.010,1,0.4,0.00
66,scierc_bert_train,2022-03-26_16.51.52.171949,70.908820,48.639109,40.052270,72.355769,50.000000,41.666667,4,20,100,100,0.00005,0.010,1,0.4,0.10
47,scierc_bert_train,2022-03-26_17.03.07.018592,70.201085,47.878851,37.453415,71.402550,48.009101,39.135381,4,20,100,100,0.00005,0.010,1,0.4,0.20
55,scierc_bert_train,2022-03-26_17.14.21.991016,69.861329,48.046050,37.443295,72.097264,48.697622,38.731597,4,20,100,100,0.00005,0.000,1,0.4,0.00


In [32]:
df = argDF.loc[argDF.label == 'ade_bert_train'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase','model_path'])
pd.DataFrame(df).to_csv('hyperparm_sel_ade_bert_train.csv')
df.sort_values('runID')

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,eval_batch_size,rel_filter_threshold,prop_drop
28,ade_bert_train,2022-03-26_20.32.43.103947,68.690618,47.655650,47.501922,69.508197,47.655650,47.501922,4,20,100,100,0.00002,0.02,1,0.4,0.0
34,ade_bert_train,2022-03-26_20.43.55.431522,89.979362,80.088170,80.088170,89.677419,80.088170,80.088170,4,20,100,100,0.00002,0.02,1,0.4,0.0
38,ade_bert_train,2022-03-26_21.06.56.446865,90.377620,81.885125,81.885125,90.027701,81.885125,81.885125,4,20,100,100,0.00002,0.02,1,0.4,0.0
14,ade_bert_train,2022-03-26_21.29.09.436457,89.890072,79.407407,79.407407,89.556377,79.407407,79.407407,4,20,100,100,0.00002,0.02,1,0.4,0.1
32,ade_bert_train,2022-03-26_21.51.25.634438,91.060012,83.272462,83.272462,90.708372,83.272462,83.272462,4,20,100,100,0.00002,0.02,1,0.4,0.1
36,ade_bert_train,2022-03-26_22.13.30.614231,89.822389,79.117647,79.117647,89.502762,79.117647,79.117647,4,20,100,100,0.00002,0.02,1,0.4,0.2
27,ade_bert_train,2022-03-26_22.35.50.323202,90.854316,82.062455,82.062455,90.508941,82.062455,82.062455,4,20,100,100,0.00002,0.02,1,0.4,0.2
29,ade_bert_train,2022-03-26_22.58.03.803951,90.397349,81.263777,81.116826,90.036900,81.263777,81.116826,4,20,100,100,0.00002,0.01,1,0.4,0.0
2,ade_bert_train,2022-03-26_23.29.14.685407,91.109804,82.753623,82.753623,90.742438,82.753623,82.753623,4,20,100,100,0.00005,0.01,1,0.4,0.2
19,ade_bert_train,2022-03-26_23.52.34.880175,89.631107,79.066375,78.920496,89.234998,79.066375,78.920496,4,20,100,100,0.00005,0.00,1,0.4,0.0


In [33]:
df = argDF.loc[argDF.label == 'ade_rob_train'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase'])
pd.DataFrame(df).to_csv('hyperparm_sel_ade_rob_train.csv')
df.sort_values('runID')

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,model_path,eval_batch_size,rel_filter_threshold,prop_drop
633,ade_rob_train,2022-03-27_13.08.51.966444,89.590266,79.253410,79.253410,89.192886,79.253410,79.253410,4,20,100,100,0.00001,0.02,roberta-base,1,0.4,0.1
635,ade_rob_train,2022-03-27_13.36.34.228031,90.492069,81.167883,81.167883,90.188853,81.167883,81.167883,4,20,100,100,0.00001,0.02,allenai/biomed_roberta_base,1,0.4,0.1
654,ade_rob_train,2022-03-27_14.04.06.615224,89.408995,79.424460,79.424460,88.939463,79.424460,79.424460,4,20,100,100,0.00001,0.02,roberta-base,1,0.4,0.2
634,ade_rob_train,2022-03-27_14.30.13.860818,90.749850,82.137628,82.137628,90.423573,82.137628,82.137628,4,20,100,100,0.00001,0.02,allenai/biomed_roberta_base,1,0.4,0.2
641,ade_rob_train,2022-03-27_14.55.04.366057,89.281876,78.832117,78.832117,88.899125,78.832117,78.832117,4,20,100,100,0.00001,0.01,roberta-base,1,0.4,0.1
642,ade_rob_train,2022-03-27_15.20.37.948485,91.140216,81.931236,81.931236,90.791897,81.931236,81.931236,4,20,100,100,0.00001,0.01,allenai/biomed_roberta_base,1,0.4,0.1
647,ade_rob_train,2022-03-27_15.46.16.096036,90.453722,81.159420,81.159420,90.045662,81.159420,81.159420,4,20,100,100,0.00001,0.01,roberta-base,1,0.4,0.2
648,ade_rob_train,2022-03-27_16.11.53.873590,90.883782,81.410727,81.410727,90.548640,81.410727,81.410727,4,20,100,100,0.00001,0.01,allenai/biomed_roberta_base,1,0.4,0.2
651,ade_rob_train,2022-03-27_16.37.33.630301,90.149850,81.129272,81.129272,89.795918,81.129272,81.129272,4,20,100,100,0.00003,0.02,roberta-base,1,0.4,0.1
652,ade_rob_train,2022-03-27_17.03.21.097864,90.730959,81.355932,81.355932,90.397045,81.355932,81.355932,4,20,100,100,0.00003,0.02,allenai/biomed_roberta_base,1,0.4,0.1


In [34]:
df = argDF.loc[argDF.label == 'ade_elec_train'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase'])
pd.DataFrame(df).to_csv('hyperparm_sel_ade_elec_train.csv')
df.sort_values('runID')

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,model_path,eval_batch_size,rel_filter_threshold,prop_drop
601,ade_elec_train,2022-03-28_08.30.15.302246,89.040705,78.143914,78.143914,88.633348,78.143914,78.143914,4,20,100,100,0.00003,0.02,google/electra-base-discriminator,1,0.4,0.1
592,ade_elec_train,2022-03-28_08.52.24.200104,90.385712,79.682330,79.682330,89.973615,79.682330,79.682330,4,20,100,100,0.00003,0.02,kamalkraj/bioelectra-base-discriminator-pubmed,1,0.4,0.1
591,ade_elec_train,2022-03-28_09.15.25.384607,89.576207,78.642715,78.642715,89.133304,78.642715,78.642715,4,20,100,100,0.00003,0.02,google/electra-base-discriminator,1,0.4,0.2
612,ade_elec_train,2022-03-28_09.39.15.694578,90.477982,79.409792,79.409792,90.101188,79.409792,79.409792,4,20,100,100,0.00003,0.02,kamalkraj/bioelectra-base-discriminator-pubmed,1,0.4,0.2
600,ade_elec_train,2022-03-28_10.04.02.063770,89.304164,78.523490,78.523490,88.888889,78.523490,78.523490,4,20,100,100,0.00003,0.01,google/electra-base-discriminator,1,0.4,0.1
604,ade_elec_train,2022-03-28_10.29.53.619221,90.608339,80.597015,80.597015,90.234202,80.597015,80.597015,4,20,100,100,0.00003,0.01,kamalkraj/bioelectra-base-discriminator-pubmed,1,0.4,0.1
610,ade_elec_train,2022-03-28_10.54.23.457476,89.571945,78.025693,78.025693,89.125610,78.025693,78.025693,4,20,100,100,0.00003,0.01,google/electra-base-discriminator,1,0.4,0.2
595,ade_elec_train,2022-03-28_11.16.56.892174,90.809247,81.019450,81.019450,90.427878,81.019450,81.019450,4,20,100,100,0.00003,0.01,kamalkraj/bioelectra-base-discriminator-pubmed,1,0.4,0.2
590,ade_elec_train,2022-03-28_11.40.36.811485,89.369785,78.882834,78.882834,88.967656,78.882834,78.882834,4,20,100,100,0.00005,0.02,google/electra-base-discriminator,1,0.4,0.1
613,ade_elec_train,2022-03-28_12.03.19.494299,90.838738,81.196013,81.196013,90.461538,81.196013,81.196013,4,20,100,100,0.00005,0.02,kamalkraj/bioelectra-base-discriminator-pubmed,1,0.4,0.1


In [35]:
df = argDF.loc[argDF.label == 'docRed_bert_train_lvl'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase'])
pd.DataFrame(df).to_csv('hyperparm_sel_docred_bert_train_lvl.csv')
df.sort_values('runID')

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,model_path,eval_batch_size,rel_filter_threshold,prop_drop
629,docRed_bert_train_lvl,2022-04-02_15.54.11.598914,87.519975,28.854797,28.339956,88.564444,57.767607,57.411645,8,20,200,200,0.00005,0.10,bert-base-cased,1,0.5,0.2
616,docRed_bert_train_lvl,2022-04-03_09.53.29.305955,87.193314,27.923648,26.971501,88.305489,57.667442,57.201965,12,20,200,200,0.00008,0.10,bert-base-cased,1,0.5,0.2
619,docRed_bert_train_lvl,2022-04-03_12.41.38.402599,81.977430,12.704020,12.277744,83.153348,54.353360,53.589613,12,30,100,100,0.00010,0.10,bert-base-cased,12,0.5,0.2
624,docRed_bert_train_lvl,2022-04-03_13.41.39.227964,84.554998,17.082534,16.696261,85.425879,56.681271,56.338992,12,30,150,100,0.00010,0.10,bert-base-cased,12,0.5,0.2
625,docRed_bert_train_lvl,2022-04-03_14.13.28.193946,84.628137,29.030721,27.335187,85.676109,57.821605,56.813088,12,30,200,100,0.00010,0.10,bert-base-cased,12,0.5,0.2
626,docRed_bert_train_lvl,2022-04-03_15.16.37.215988,86.609358,33.761825,33.173605,87.555185,60.850281,60.471657,12,30,200,100,0.00008,0.10,bert-base-cased,12,0.5,0.2
627,docRed_bert_train_lvl,2022-04-03_18.02.08.888896,86.089758,32.297164,31.953536,87.375487,61.447252,61.225621,12,30,200,200,0.00008,0.10,bert-base-cased,12,0.5,0.2
615,docRed_bert_train_lvl,2022-04-03_20.17.39.406912,87.039750,33.252674,32.569559,88.185301,59.654631,59.065934,12,30,200,200,0.00003,0.01,bert-base-cased,12,0.5,0.1
620,docRed_bert_train_lvl,2022-04-03_22.23.39.487412,87.127141,32.448905,32.116244,88.272639,58.803369,58.558559,12,30,200,200,0.00003,0.01,bert-base-cased,12,0.5,0.2
621,docRed_bert_train_lvl,2022-04-04_00.28.25.135008,86.851320,33.148832,32.421541,88.058373,58.970588,58.578431,12,30,200,200,0.00003,0.00,bert-base-cased,12,0.5,0.1


In [36]:
df = argDF.loc[argDF.label == 'docRed_rob_train_lvl'][['label','runID','ner_f1_macro','rel_f1_macro','rel_nec_f1_macro', 'ner_f1_micro','rel_f1_micro',	'rel_nec_f1_micro']+varCols[:lmt]].drop(columns=['config','model_type','index','train_path','valid_path','types_path','final_eval','lowercase'])
pd.DataFrame(df).to_csv('hyperparm_sel_docred_rob_train_lvl.csv')
df.sort_values('runID')

,label,runID,ner_f1_macro,rel_f1_macro,rel_nec_f1_macro,ner_f1_micro,rel_f1_micro,rel_nec_f1_micro,train_batch_size,epochs,neg_entity_count,neg_relation_count,lr,weight_decay,model_path,eval_batch_size,rel_filter_threshold,prop_drop
214,docRed_rob_train_lvl,2022-04-04_20.29.00.322642,86.264739,22.630432,22.313493,87.547632,59.210526,58.947368,8,30,200,200,0.00003,0.1,roberta-base,1,0.5,0.2
207,docRed_rob_train_lvl,2022-04-04_23.10.02.989790,86.133436,20.756293,20.330532,87.367194,59.155844,58.766234,8,30,200,200,0.00003,0.1,roberta-base,1,0.5,0.1
212,docRed_rob_train_lvl,2022-04-05_01.50.51.744269,86.170738,20.315200,19.766956,87.546156,58.773087,58.311346,8,30,200,200,0.00003,0.0,roberta-base,1,0.5,0.2
209,docRed_rob_train_lvl,2022-04-05_04.27.46.671195,85.937644,22.451591,21.825833,87.174267,59.284979,58.899177,8,30,200,200,0.00003,0.0,roberta-base,1,0.5,0.1
213,docRed_rob_train_lvl,2022-04-05_07.04.03.787270,85.528721,24.754653,24.117967,86.872692,58.887810,58.402137,8,30,200,200,0.00005,0.1,roberta-base,1,0.5,0.2
210,docRed_rob_train_lvl,2022-04-05_09.47.37.728639,85.787924,25.393303,23.968945,87.019441,58.645096,58.076340,8,30,200,200,0.00005,0.1,roberta-base,1,0.5,0.1
215,docRed_rob_train_lvl,2022-04-05_12.47.34.061412,85.744874,22.928500,22.476633,87.067653,59.126782,58.872200,8,30,200,200,0.00005,0.0,roberta-base,1,0.5,0.2
208,docRed_rob_train_lvl,2022-04-05_15.37.34.269938,85.437263,24.984872,24.471271,86.878950,57.232481,56.758996,8,30,200,200,0.00005,0.0,roberta-base,1,0.5,0.1
211,docRed_rob_train_lvl,2022-04-05_18.32.01.569345,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8,30,200,200,0.00008,0.1,roberta-base,1,0.5,0.2
